# New peepholes!

## Prime prove

In [ ]:
import os
import torch
import numpy as np
from tensordict import TensorDict
from tensordict import MemoryMappedTensor as MMT

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# python stuff
from pathlib import Path as Path
from numpy.random import randint

# Our stuff
from datasets.cifar import Cifar
from models.model_wrap import ModelWrap 
from peepholes.peepholes import Peepholes
from peepholes.svd_peepholes import peep_matrices_from_svds as parser_fn

# torch stuff
import torch
from torchvision.models import vgg16, VGG16_Weights

In [ ]:
import pickle
#import datetime
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans

In [ ]:
from clustering.clustering import *

In [ ]:
# TODO: cambiare i path (anche nei moduli)
p_dir = os.path.join('../data/peepholes')
svd_dir = os.path.join('../data/svds')

## Load SVD 

In [ ]:
file_path = os.path.join(svd_dir, 'svds')
svds = TensorDict.load_memmap(file_path)

### Singular Values visualization

In [ ]:
fig, axs = plt.subplots()
for i, layer in enumerate(svds.keys()):
    axs.plot(svds[layer]['s'], label=layer, alpha=0.7)
axs.set_xlim(-2, 300)
axs.legend()
plt.tight_layout()

## Load dataset

In [ ]:
#--------------------------------
# Dataset 
#--------------------------------
# model parameters
dataset = 'CIFAR10' 
seed = 29
bs = 64

ds_path = f'/srv/newpenny/dataset/{dataset}'
ds = Cifar(data_path=ds_path,
           dataset=dataset)

ds.load_data(
        batch_size = bs,
        data_kwargs = {'num_workers': 4, 'pin_memory': True},
        seed = seed,
        )

## Load model

In [ ]:
use_cuda = torch.cuda.is_available()
cuda_index = torch.cuda.device_count() - 2
device = torch.device(f"cuda:{cuda_index}" if use_cuda else "cpu")
print(f"Using {device} device")

In [ ]:
pretrained = True
model_dir = '/srv/newpenny/XAI/LM/models'
model_name = f'vgg16_pretrained={pretrained}_dataset={dataset}-'\
f'augmented_policy=CIFAR10_bs={bs}_seed={seed}.pth'


nn = vgg16(weights=VGG16_Weights.IMAGENET1K_V1)
in_features = 4096
num_classes = len(ds.get_classes()) 
nn.classifier[-1] = torch.nn.Linear(in_features, num_classes)
model = ModelWrap(device=device)
model.set_model(model=nn, path=model_dir, name=model_name, verbose=True)

## Load peepholes

In [ ]:
abs_path = '/srv/newpenny/XAI/generated_data'

In [ ]:
phs_name = 'peepholes'
phs_dir = os.path.join(abs_path, 'peepholes')
peepholes = Peepholes(
        path = phs_dir,
        name = phs_name,
        )
loaders = ds.get_dataset_loaders()


In [ ]:
# copy dataset to peepholes dataset
peepholes.get_peep_dataset(
        loaders = loaders,
        verbose = True
        ) 

In [ ]:
ph_dl = peepholes.get_dataloaders(batch_size=128, verbose=True)

In [ ]:
layers_dict = {'classifier': [0, 3],
               'features': [24, 26, 28]}
model.set_target_layers(target_layers=layers_dict, verbose=True)
print('target layers: ', model.get_target_layers().keys()) 

In [ ]:
layers_list = list(model.get_target_layers().keys())

In [ ]:
available_layers = list(next(iter(ph_dl['train']))['peepholes'].keys())
#list(next(iter(ph_dl['train']))['peepholes'].keys())

## Clustering class

* fit on training data
    * can obtain k from the data shape 
* the Class can provide up to the output_peephole per layer. then we will use dedicated functions to get:
    * quantities per single layer (max, entropy)
    * combined scores with multiple layers (simple weights and exponentials)

In [ ]:
clust_kwargs = {}

layer = 'classifier-3'

clust_kwargs[layer]= {'algorithm': 'kmeans',
                      'k': 100,
                      'n': num_classes,
                       }


In [ ]:
class Clustering: # quella buona
    def __init__(self, algorithm, k, n_clusters, seed=42, 
                 base_dir='clustering'):
        self.algorithm = algorithm  
        self.k = k  
        self.n_clusters = n_clusters
        self.seed = seed
        self.base_dir = f'/srv/newpenny/XAI/generated_data/{base_dir}'

        self._fitted_model = None
        self._cluster_assignments = None        # cluster assignments from the model
        self._cluster_centers = None            # cluster centers from the model
        self._cluster_covariances = None        # cluster covariances from the model
        self._empirical_posteriors = None       # empirical posteriors (P(g, c))

    def fit(self, core_vectors, labels=None):
        '''
        Perform clustering on the training core_vectors of a specific layer.
        
        Args:
        - core_vectors (Tensor): Ex-"peepholes" with reduced dimension (n_samples, k)
        - labels (Tensor): Labels for empirical posteriors computation (optional)
        '''
        #if self.algorithm == 'gmm':
        #    model = GaussianMixture(n_components=self.n_clusters, random_state=self.seed)
        #    model.fit(core_vectors)
        #    self._cluster_assignments = model.predict(core_vectors)
        #    self._cluster_centers = model.means_
        #    self._cluster_covariances = model.covariances_
        #    self._fitted_model = model
        #    
        #elif self.algorithm == 'kmeans':
        #    model = KMeans(n_clusters=self.n_clusters, random_state=self.seed)
        #    model.fit(core_vectors)
        #    self._cluster_assignments = model.predict(core_vectors)
        #    self._cluster_centers = model.cluster_centers_
        #    self._fitted_model = model
#
        ## compute empirical posteriors if labels are provided
        #if labels is not None:
        #    self.compute_empirical_posteriors(labels)
        if self.algorithm == 'gmm':
            model = GaussianMixture(n_components=self.n_clusters, random_state=self.seed)
            model.fit(core_vectors)
            self._cluster_assignments = model.predict(core_vectors)
            self._cluster_centers = model.means_
            self._cluster_covariances = model.covariances_
            self._fitted_model = model
    
        elif self.algorithm == 'kmeans':
            model = KMeans(n_clusters=self.n_clusters, random_state=self.seed)
            model.fit(core_vectors)
            self._cluster_assignments = model.predict(core_vectors)
            self._cluster_centers = model.cluster_centers_
            self._fitted_model = model
    
        # Check if clustering was successful
        if self._cluster_assignments is None:
            raise ValueError("Clustering failed. No assignments were generated.")
    
        # Compute empirical posteriors if labels are provided
        if labels is not None:
            self.compute_empirical_posteriors(labels)

    def compute_empirical_posteriors(self, labels):
        '''
        Compute the empirical posterior matrix P, where P(g, c) is the probability
        that a sample assigned to cluster g belongs to class c.

        Args:
        - labels (Tensor): True class labels for the samples (n_samples, )
        '''
        n_samples = len(labels)
        n_classes = len(torch.unique(labels))
        
        # initialize matrix to count occurrences of (cluster g, class c) pairs
        P_counts = torch.zeros(self.n_clusters, n_classes)

        # count occurrences of (cluster g, class c) pairs
        for i in range(n_samples):
            c = int(labels[i].item())  # true class label
            g = int(self._cluster_assignments[i])  # cluster assignment
            P_counts[g, c] += 1

        # normalize to get empirical posteriors
        P_empirical = P_counts / P_counts.sum(dim=1, keepdim=True)

        # nandle potential division by zero
        P_empirical = torch.nan_to_num(P_empirical)  # replace NaN with 0

        self._empirical_posteriors = P_empirical

    def cluster_probabilities(self, core_vectors):
        '''
        Get cluster probabilities for the provided core_vectors based on the fitted model.
        
        Args:
        - core_vectors (Tensor): Peepholes with reduced dimension (n_samples, k)
        
        Returns:
        - cluster_probs (Tensor): Probabilities for each cluster (n_samples, n_clusters)
        '''
        if self.algorithm == 'gmm':
            return self._fitted_model.predict_proba(core_vectors)  # (n_samples, n_clusters)
        elif self.algorithm == 'kmeans':
            # get distances to each cluster center
            distances = self._fitted_model.transform(core_vectors)
            distances = torch.tensor(distances)
            # convert distances to probabilities (soft assignment)
            cluster_probs = torch.exp(-distances ** 2 / (2 * (distances.std() ** 2)))  # Gaussian-like softmax
            cluster_probs = cluster_probs / cluster_probs.sum(dim=1, keepdim=True)  # normalize to probabilities
            return cluster_probs

    def map_clusters_to_classes(self, core_vectors):
        '''
        Map the cluster probabilities to class probabilities using empirical posteriors.
        
        Args:
        - cluster_probs (Tensor): Probabilities for each cluster (n_samples, n_clusters)
        
        Returns:
        - class_probs (Tensor): Probabilities for each class (n_samples, n_classes)
        '''
        if self._empirical_posteriors is None:
            raise RuntimeError('Please run compute_empirical_posteriors() first.')

        cluster_probs = self.cluster_probabilities(core_vectors)
        cluster_probs = torch.tensor(cluster_probs, dtype=torch.float32)
        
        class_probs = torch.matmul(cluster_probs, self._empirical_posteriors)  # shape: (n_samples, n_classes)
        class_probs = class_probs / class_probs.sum(dim=1, keepdim=True) # aka the new peepholes
        return class_probs

    # confidence scores ---------------------
    #def get_confidence_scores(self, class_probs, score_type="max"):
    #    '''
    #    Compute confidence scores (either max or entropy), save them, or load if they already exist.
    #    
    #    Args:
    #    - class_probs (Tensor): Probabilities for each class (n_samples, n_classes)
    #    - score_type (str): Either 'max' or 'entropy' to specify the type of confidence score.
    #    
    #    Returns:
    #    - confidence_scores (Tensor): The computed confidence scores (n_samples, )
    #    '''
    #    confidence_dir = os.path.join(self.base_dir, 'confidence_scores')
    #    filepath = self.construct_filepath(prefix=f"cs={score_type}", suffix="pkl", dir_path=confidence_dir)
    #    
    #    if os.path.exists(filepath):
    #        print(f"Confidence scores ({score_type}) already exist at {filepath}. Loading...")
    #        with open(filepath, 'rb') as f:
    #            confidence_scores = pickle.load(f)
    #        return confidence_scores
    #
    #    if score_type == "max":
    #        confidence_scores = torch.max(class_probs, dim=1).values
    #    elif score_type == "entropy":
    #        # entropy: -sum(p * log(p)) across classes (axis 1)
    #        entropy = -torch.sum(class_probs * torch.log(class_probs + 1e-12), dim=1)
    #        confidence_scores = entropy
    #    else:
    #        raise ValueError(f"Invalid score_type: {score_type}. Use 'max' or 'entropy'.")
    #
    #    os.makedirs(confidence_dir, exist_ok=True)
    #    with open(filepath, 'wb') as f:
    #        pickle.dump(confidence_scores, f)
    #    print(f'Confidence scores ({score_type}) saved to {filepath}')
    #
    #    return confidence_scores
    def get_confidence_scores(self, class_probs, split='train', score_type='max', save=False):
        #confidence_dir = os.path.join(self.base_dir, 'confidence_scores')
        #os.makedirs(confidence_dir, exist_ok=True)
#
        #filename = "_".join([f'cs={score_type}', self.construct_filename()])
        #filepath = os.path.join(confidence_dir, split, filename)
#
        #if os.path.exists(filepath):
        #    print(f"Confidence scores ({score_type}) already exist at {filepath}. Loading...")
        #    with open(filepath, 'rb') as f:
        #        confidence_scores = pickle.load(f)
        #    return confidence_scores

        if score_type == "max":
            confidence_scores = torch.max(class_probs, dim=1).values
        elif score_type == "entropy":
            entropy = -torch.sum(class_probs * torch.log(class_probs + 1e-12), dim=1)
            confidence_scores = entropy
        else:
            raise ValueError(f"Invalid score_type: {score_type}. Use 'max' or 'entropy'.")
        #if save:
        #    with open(filepath, 'wb') as f:
        #        pickle.dump(confidence_scores, f)
        #    print(f'Confidence scores ({score_type}) saved to {filepath}')

        return confidence_scores



    # save/load results ---------------------
    def save_cluster_results(self, filepath=None):
        """
        Save the clustering results (assignments, centers, covariances) to a file.
        """
        if filepath is None:
            filepath = self.construct_filepath(suffix='pkl') 

        data = {
            'assignments': self._cluster_assignments,
            'centers': self._cluster_centers,
            'k': self.k,
        }

        if self.algorithm == 'gmm':
            data['covariances'] = self._cluster_covariances

        os.makedirs(os.path.dirname(filepath), exist_ok=True)

        with open(filepath, 'wb') as f:
            pickle.dump(data, f)

        # print(f'Clustering results saved to {filepath}')

    def load_cluster_results(self, filepath=None):
        """
        Load clustering results from a file.
        """
        if filepath is None:
            filepath = self.construct_filepath(suffix='pkl') 

        try:
            with open(filepath, 'rb') as f:
                results = pickle.load(f)
            self._cluster_assignments = results['assignments']
            self._cluster_centers = results['centers']
            self.k = results.get('k', self.k)

            if self.algorithm == 'gmm' and 'covariances' in results:
                self._cluster_covariances = results['covariances']

            print(f'Clustering results loaded from {filepath}')
        except FileNotFoundError:
            print(f"File {filepath} not found")
        except Exception as e:
            print(f"An error occurred while loading clustering results: {e}")

    def construct_filepath(self, suffix='pkl', **extra_kwargs):
        '''
        Constructs a file path for saving or loading clustering results based on attributes.
        Combines the base directory, attributes, and extra arguments into the file name.
        '''
 
        dir_path = self.base_dir
        os.makedirs(dir_path, exist_ok=True)
        filename = self.construct_filename(suffix=suffix, **extra_kwargs)
        return os.path.join(dir_path, filename)

    def construct_filename(self, suffix='pkl', **extra_kwargs):
        '''
        Constructs a detailed filename for saving clustering results,
        using class attributes and any extra keyword arguments passed.
        '''

        filename_kwargs = self.generate_kwargs_from_attrs()
        filename_kwargs.update(extra_kwargs)

        filename_parts = [f"{k}={v}" for k, v in filename_kwargs.items()]
        filename = "_".join(filename_parts) + f".{suffix}"
        
        return filename

    def generate_kwargs_from_attrs(self):
        '''
        Generate a dictionary of current class attributes and their values.
        This can be used for constructing filenames or passing arguments.
        '''
        attrs = {
            'algorithm': self.algorithm,
            'k': self.k,
            'n_clusters': self.n_clusters,
            'seed': self.seed
        }
        
        return attrs


In [ ]:
class Clustering:
    def __init__(self, algorithm, k, n_clusters, seed=42, base_dir='clustering'):
        self.algorithm = algorithm  
        self.k = k  
        self.n_clusters = n_clusters
        self.seed = seed
        self.base_dir = f'../data/{base_dir}'

        self._fitted_model = None
        self._cluster_assignments = None        # cluster assignments from the model
        self._cluster_centers = None            # cluster centers from the model
        self._cluster_covariances = None        # cluster covariances from the model
        self._empirical_posteriors = None       # empirical posteriors (P(g, c))

    def fit(self, core_vectors, labels=None):
        '''
        Perform clustering on the core_vectors of a specific layer.
        
        Args:
        - core_vectors (Tensor): Ex-"peepholes" with reduced dimension (n_samples, k)
        - labels (Tensor): Labels for empirical posteriors computation (optional)
        '''
        if self.algorithm == 'gmm':
            model = GaussianMixture(n_components=self.n_clusters, random_state=self.seed)
            model.fit(core_vectors)
            self._cluster_assignments = model.predict(core_vectors)
            self._cluster_centers = model.means_
            self._cluster_covariances = model.covariances_
            self._fitted_model = model
            
        elif self.algorithm == 'kmeans':
            model = KMeans(n_clusters=self.n_clusters, random_state=self.seed)
            model.fit(core_vectors)
            self._cluster_assignments = model.predict(core_vectors)
            self._cluster_centers = model.cluster_centers_
            self._fitted_model = model

        # compute empirical posteriors if labels are provided
        if labels is not None:
            self.compute_empirical_posteriors(labels)

    def compute_empirical_posteriors(self, labels):
        '''
        Compute the empirical posterior matrix P, where P(g, c) is the probability
        that a sample assigned to cluster g belongs to class c.

        Args:
        - labels (Tensor): True class labels for the samples (n_samples, )
        '''
        n_samples = len(labels)
        n_classes = len(torch.unique(labels))
        
        # initialize matrix to count occurrences of (cluster g, class c) pairs
        P_counts = torch.zeros(self.n_clusters, n_classes)

        # count occurrences of (cluster g, class c) pairs
        for i in range(n_samples):
            c = int(labels[i].item())  # true class label
            g = int(self._cluster_assignments[i])  # cluster assignment
            P_counts[g, c] += 1

        # normalize to get empirical posteriors
        P_empirical = P_counts / P_counts.sum(dim=1, keepdim=True)

        # nandle potential division by zero
        P_empirical = torch.nan_to_num(P_empirical)  # replace NaN with 0

        self._empirical_posteriors = P_empirical

    def cluster_probabilities(self, core_vectors):
        '''
        Get cluster probabilities for the provided core_vectors based on the fitted model.
        
        Args:
        - core_vectors (Tensor): Peepholes with reduced dimension (n_samples, k)
        
        Returns:
        - cluster_probs (Tensor): Probabilities for each cluster (n_samples, n_clusters)
        '''
        if self.algorithm == 'gmm':
            return self._fitted_model.predict_proba(core_vectors)  # (n_samples, n_clusters)
        elif self.algorithm == 'kmeans':
            # get distances to each cluster center
            distances = self._fitted_model.transform(core_vectors)
            distances = torch.tensor(distances)
            # convert distances to probabilities (soft assignment)
            cluster_probs = torch.exp(-distances ** 2 / (2 * (distances.std() ** 2)))  # Gaussian-like softmax
            cluster_probs = cluster_probs / cluster_probs.sum(dim=1, keepdim=True)  # normalize to probabilities
            return cluster_probs

    def map_clusters_to_classes(self, core_vectors):
        '''
        Map the cluster probabilities to class probabilities using empirical posteriors.
        
        Args:
        - cluster_probs (Tensor): Probabilities for each cluster (n_samples, n_clusters)
        
        Returns:
        - class_probs (Tensor): Probabilities for each class (n_samples, n_classes)
        '''
        if self._empirical_posteriors is None:
            raise RuntimeError('Please run compute_empirical_posteriors() first.')

        cluster_probs = self.cluster_probabilities(core_vectors)
        cluster_probs = torch.tensor(cluster_probs, dtype=torch.float32)
        
        class_probs = torch.matmul(cluster_probs, self._empirical_posteriors)  # shape: (n_samples, n_classes)
        class_probs = class_probs / class_probs.sum(dim=1, keepdim=True) # aka the new peepholes
        return class_probs

    # save/load results ---------------------


In [ ]:
algorithm = 'gmm'
k = 20
n_clusters = int(num_classes/2)

clustering = Clustering(algorithm, k, n_clusters)

In [ ]:
# prepare data
core_vectors = {}
v_labels = {}
decisions = {}

for split in ['train', 'val', 'test']:
    first_batch = next(iter(ph_dl['train']))
    p = first_batch['peepholes']
    layer_keys = p.keys()  
    
    core_vectors[split] = {key: [] for key in layer_keys}
    v_labels[split] = []
    decisions[split] = []

    for batch in ph_dl[split]:
        
        peepholes = batch['peepholes']
        labels = batch['label']
        decision_results = batch['result']

        for layer, peephole_tensor in peepholes.items():
            batch_size, d = peephole_tensor.shape
                            
            reduced_peephole = peephole_tensor[:, :k]

            core_vectors[split][layer].append(reduced_peephole)
        v_labels[split].append(labels)
        decisions[split].append(decision_results.bool())
        
    for layer in core_vectors[split]:
        core_vectors[split][layer] = torch.cat(core_vectors[split][layer], dim=0) 
    
    v_labels[split] = torch.cat(v_labels[split], dim=0)
    decisions[split] = torch.cat(decisions[split], dim=0)

In [ ]:
vars(clustering)

In [ ]:
# fit clustering model
split = 'train'
layer = 'classifier.0'
labels = v_labels[split]

clustering.fit(core_vectors[split][layer], labels)

In [ ]:
plt.imshow(clustering._empirical_posteriors)

In [ ]:
max_scores = {}
entropy_scores = {}

for split in ['train', 'val']:
    class_probs = clustering.map_clusters_to_classes(core_vectors[split][layer])
    _max = clustering.get_confidence_scores(class_probs, split=split, score_type='max')
    _entropy = clustering.get_confidence_scores(class_probs, split=split, score_type='entropy')

    max_scores[split] = _max
    entropy_scores[split] = _entropy

### structure without tensordict

In [ ]:
from tqdm import tqdm

In [ ]:
k_list = [20]
n_clusters_list = [50]

# select algorithm
algorithm = 'gmm'

# loop over k and n_clusters
for k in k_list:
    # prepare data
    core_vectors = {}
    v_labels = {}
    decisions = {}
    print('Preparing data')
    for split in ['train', 'val']: #, 'test']:
        first_batch = next(iter(ph_dl['train']))
        p = first_batch['peepholes']
        layer_keys = p.keys() # so we can automatically loop over available layers 
        
        core_vectors[split] = {key: [] for key in layer_keys}
        v_labels[split] = []
        decisions[split] = []
    
        for batch in ph_dl[split]:
            
            peepholes = batch['peepholes']
            labels = batch['label']
            decision_results = batch['result']
    
            for layer, peephole_tensor in peepholes.items():
                batch_size, d = peephole_tensor.shape
                                
                reduced_peephole = peephole_tensor[:, :k]
    
                core_vectors[split][layer].append(reduced_peephole)
            v_labels[split].append(labels)
            decisions[split].append(decision_results.bool())
            
        for layer in core_vectors[split]:
            core_vectors[split][layer] = torch.cat(core_vectors[split][layer], dim=0) 
        
        v_labels[split] = torch.cat(v_labels[split], dim=0)
        decisions[split] = torch.cat(decisions[split], dim=0)
    print('Data is ready')
    # clustering init and fit for each layer
    for n_clusters in tqdm(n_clusters_list):
        split = 'train'
        
        clustering = {}
        for layer in layer_keys:
            
            clustering[layer] = Clustering(algorithm, k, n_clusters)
            labels = v_labels[split]
            clustering[layer].fit(core_vectors[split][layer], labels)
    
        # use train split (get scores)
        max_scores = {}
        entropy_scores = {}
        split = 'train'
        class_probs = clustering[layer].map_clusters_to_classes(core_vectors[split][layer])
        _max = clustering[layer].get_confidence_scores(class_probs, split=split, score_type='max')
        _entropy = clustering[layer].get_confidence_scores(class_probs, split=split, score_type='entropy')
    
        max_scores[split] = _max
        entropy_scores[split] = _entropy
    
        # use val split
        split = 'val'
        class_probs = clustering[layer].map_clusters_to_classes(core_vectors[split][layer])
        _max = clustering[layer].get_confidence_scores(class_probs, split=split, score_type='max')
        _entropy = clustering[layer].get_confidence_scores(class_probs, split=split, score_type='entropy')
    
        max_scores[split] = _max
        entropy_scores[split] = _entropy

### structure with tensordict

Hopefully will look something like:
```python
all_scores = {
    20: {  # k value
        50: {  # n_clusters value
            'train': {
                'layer1': {
                    'max': tensor([...]),
                    'entropy': tensor([...])
                },
                'layer2': {
                    'max': tensor([...]),
                    'entropy': tensor([...])
                }
            },
            'val': {
                'layer1': {
                    'max': tensor([...]),
                    'entropy': tensor([...])
                },
                'layer2': {
                    'max': tensor([...]),
                    'entropy': tensor([...])
                }
            }
        }
    }
}


In [ ]:
k_list = [20]
n_clusters_list = [50]

# select algorithm
algorithm = 'gmm'

# init results container
#all_scores = TensorDict()
all_scores = TensorDict({}, batch_size=[])

# loop over k and n_clusters
for k in k_list:
    all_scores.set(str(k), TensorDict({}, batch_size=[]))  # set for k
    # prepare data
    print('Preparing data')
    core_vectors = {}
    v_labels = {}
    decisions = {}

    for split in ['train', 'val']: #, 'test']:
        first_batch = next(iter(ph_dl['train']))
        p = first_batch['peepholes']
        layer_keys = p.keys() # so we can automatically loop over available layers 
        
        core_vectors[split] = {key: [] for key in layer_keys}
        v_labels[split] = []
        decisions[split] = []
    
        for batch in ph_dl[split]:
            
            peepholes = batch['peepholes']
            labels = batch['label']
            decision_results = batch['result']
    
            for layer, peephole_tensor in peepholes.items():
                batch_size, d = peephole_tensor.shape
                                
                reduced_peephole = peephole_tensor[:, :k]
    
                core_vectors[split][layer].append(reduced_peephole)
            v_labels[split].append(labels)
            decisions[split].append(decision_results.bool())
            
        for layer in core_vectors[split]:
            core_vectors[split][layer] = torch.cat(core_vectors[split][layer], dim=0) 
        
        v_labels[split] = torch.cat(v_labels[split], dim=0)
        decisions[split] = torch.cat(decisions[split], dim=0)
    print('Data is ready')
    # clustering init and fit for each layer
    for n_clusters in tqdm(n_clusters_list):

        #all_scores[k] = {n_clusters: {}}
        all_scores[str(k)].set(str(n_clusters), TensorDict({
            'train': TensorDict({}, batch_size=[]),
            'val': TensorDict({}, batch_size=[])
        }, batch_size=[]))
        
        # fit clustering model for each layer on train split
        clustering = {}
        for layer in tqdm(layer_keys):
            clustering[layer] = Clustering(algorithm, k, n_clusters)
            labels = v_labels['train']  # Use the 'train' split labels
            clustering[layer].fit(core_vectors['train'][layer], labels)

        # compute max and entropy scores for both 'train' and 'val'
        for split in ['train', 'val']:
            all_scores[str(k)][str(n_clusters)][split] = {}
            for layer in layer_keys:
                class_probs = clustering[layer].map_clusters_to_classes(core_vectors[split][layer])

                _max = clustering[layer].get_confidence_scores(class_probs, split=split, score_type='max')
                _entropy = clustering[layer].get_confidence_scores(class_probs, split=split, score_type='entropy')

                all_scores[str(k)][str(n_clusters)][split][layer] = {
                    'max': _max,
                    'entropy': _entropy
                }
        

In [ ]:
all_scores[str(k)][str(n_clusters)]['val']['classifier.0']['entropy']

In [ ]:
# get right filepname and save td + metadata

In [ ]:
abs_path = '/srv/newpenny/XAI/generated_data/clustering'
res_dir = 'confidence_scores'
res_path = os.path.join(abs_path, res_dir)

In [ ]:
dnn_model = 'vgg16'
res_suffix = '.pth'
res_filename = f'algorithm={algorithm}_dataset={dataset}_dnn={dnn_model}'
torch.save(all_scores, os.path.join(res_path, res_filename + res_suffix))

In [ ]:
metadata = {
    'k_values': k_list,
    'n_clusters': n_clusters_list,
    'layers': list(layer_keys)
}

In [ ]:
metadata

In [ ]:
import json

In [ ]:
from clustering.clustering import prepare_data, compute_scores

In [ ]:
layer = 'classifier.0'
k = 20
data = prepare_data(ph_dl, [layer], k)

In [ ]:
len(data['core_vectors']['train'][layer])

In [ ]:
len(data['true_labels']['train'])

In [ ]:
kk = Clustering('gmm', 20, 50)

In [ ]:
kk.fit(data['core_vectors']['train'][layer], data['true_labels']['train'])

In [ ]:
kk.compute_empirical_posteriors(data['true_labels']['train'])

In [ ]:
core_vectors = data['core_vectors']
v_labels = data['true_labels']
decisions = data['decisions']

In [ ]:
available_layers = list(next(iter(ph_dl['train']))['peepholes'].keys())

In [ ]:
layers_list = available_layers
compute_scores(k, n_clusters, layers_list, data, all_scores, metadata)

In [ ]:
data['core_vectors'].keys()

In [ ]:
#for split in data['core_vectors'].keys():
for layer in [layer]:
    compute_scores(20, 50, 'gmm', [layer], data, all_scores=None, metadata=None)

In [ ]:
layer_keys = layers_list

In [ ]:
# init params
k_list = [20]
n_clusters_list = [50, 100]
algorithm = 'gmm'

# check for existing results or init results container
res_suffix = '.pth'
res_filename = f'algorithm={algorithm}_dataset={dataset}_dnn={dnn_model}'
tensor_dict_path = os.path.join(res_path, res_filename + res_suffix)

meta_filename = '_'.join(['metadata', res_filename]) + '.json'
meta_path = os.path.join(res_path, meta_filename)

# check if results and metadata exist
results_exist = os.path.exists(tensor_dict_path)
metadata_exist = os.path.exists(meta_path)

if results_exist:
    print('Results already present')
    all_scores = torch.load(tensor_dict_path)
else:
    all_scores = TensorDict({}, batch_size=[])

if metadata_exist:
    print('Loading related metadata')
    with open(meta_path, 'r') as json_file:
        metadata = json.load(json_file)
else:
    metadata = {'k_values': [], 'n_clusters': [], 'layers': []}

# loop over k and n_clusters
for k in k_list:
    str_k = str(k) if not isinstance(k, str) else k

    if str_k not in all_scores.keys():
        all_scores.set(str_k, TensorDict({}, batch_size=[]))

    for n_clusters in n_clusters_list:
        str_n_clusters = str(n_clusters) if not isinstance(n_clusters, str) else n_clusters

        # check if the combination of k and n_clusters already exists in the scores
        if str_n_clusters in all_scores[str_k].keys():
            existing_layers = all_scores[str_k][str_n_clusters]['train'].keys()

            # check if only specific layers need to be computed
            if existing_layers and any(layer not in existing_layers for layer in layer_keys):
                # compute only the missing layers
                for layer in layer_keys:
                    if layer not in existing_layers:
                        data = prepare_data(ph_dl, [layer], k)
                        compute_scores(k, n_clusters, [layer], data, all_scores, metadata)

            else:
                print(f"Skipping k={k}, n_clusters={n_clusters} for all layers as it's already computed.")
                continue  # skip to the next n_clusters if all layers have data

        else:
            # if not existing, create the subdict for n_clusters and compute all layers
            all_scores[str_k].set(str_n_clusters, TensorDict({
                'train': TensorDict({}, batch_size=[]),
                'val': TensorDict({}, batch_size=[])
            }, batch_size=[]))
            existing_layers = []  # no existing layers yet

            # prepare data for all splits
            data = prepare_data(ph_dl, layer_keys, k)

            # compute scores for all layers and splits
            for split in data['core_vectors'].keys():
                for layer in layer_keys:
                    compute_scores(k, n_clusters, layer, all_scores, metadata)

# Save all scores and metadata after processing
torch.save(all_scores, tensor_dict_path)
with open(meta_path, 'w') as json_file:
    json.dump(metadata, json_file, indent=4)

print("Results and metadata saved.")

## TODO

* split val data into correct/wrong
* make confusion matrix for each threshold on the training scores
* do it for single layers and combined

In [ ]:
# prepare data
# if you need only labels and decision outcomes, k and layers_list are irrelevant
data = prepare_data(ph_dl, available_layers[:1], k=10)

In [ ]:
# get correct/wrong decision indices
decisions = data['decisions']
decisions

In [ ]:
# load collected scores
dataset = 'CIFAR100'
dnn_model = 'vgg16'
algorithm = 'gmm'

In [ ]:
# load existing results
# check for existing results or init results container
res_dir = 'clustering/confidence_scores'
res_path = os.path.join(abs_path, res_dir)
tensor_dict_path = os.path.join(res_path, f'algorithm={algorithm}_dataset={dataset}_dnn={dnn_model}.memmap')

if os.path.exists(tensor_dict_path):
    print('Results already present')
    all_scores = TensorDict.load_memmap(tensor_dict_path)
else:
    all_scores = TensorDict({}, batch_size=[])

# init the peephole container if not existing
new_peep_dir = 'clustering/peepholes' 
new_peep_path = os.path.join(abs_path, new_peep_dir) 
new_peep_tensor_dict_path = os.path.join(new_peep_path, f'algorithm={algorithm}_dataset={dataset}_dnn={dnn_model}.memmap')

if os.path.exists(new_peep_tensor_dict_path):
    print('New peepholes results already present')
    peephole_scores = TensorDict.load_memmap(new_peep_tensor_dict_path)
else:
    # print('Initializing peephole container')
    peephole_scores = TensorDict({}, batch_size=[])

In [ ]:
k_values_list, n_clusters_list, splits_list, layers_list = get_unique_values(all_scores)

In [ ]:
k_values_list

In [ ]:
n_clusters_list

In [ ]:
splits_list

In [ ]:
layers_list

In [ ]:
for k in k_values_list:
    for n in n_clusters_list:
        for layer in layers_list:
            for measure in ['max', 'entropy']:
                list_correct = [] 
                list_wrong = []  
 
                conf_t = all_scores[k][n]['train'][layer][measure] 
                conf_v = all_scores[k][n]['val'][layer][measure]    
            
                for i in quantiles:
                    # compute threshold
                    q = torch.quantile(conf_t, i)

                    if measure=='max':
                        # get indices where validation scores exceed the threshold
                        idx = torch.where(conf_v > q)[0]  
                    elif measure=='entropy':
                        torch.where(conf_v < q)[0]
            


In [ ]:
for j, (layer, weight) in enumerate(w_dict.items()):

    for n in num_clusters:    
        
        fig, axs = plt.subplots(1, figsize=(6, 6))
        axs.grid()
        axs.plot([0, 1],[1,0],label='ref', c='k', ls='--')
        
        for dim in dims_list: 
            
            prob_train = out_p_prob_train[layer][(dim, n)]
            prob_val = out_p_prob_val[layer][(dim, n)]
            
            if measure=='max':
                conf_t = np.max(prob_train,axis=1)
                conf_v = np.max(prob_val,axis=1)
            elif measure=='entropy':
                conf_t = H(prob_train,axis=1)
                conf_v = H(prob_val,axis=1)
            
            threshold = []
            list_true_max_ = []
            list_false_max_ = []
            
            for i in array:
            
                perc = np.quantile(conf_t, i)
                
                threshold.append(perc)
                idx = np.argwhere(conf_v>perc)[:,0]
                counter = collections.Counter(results_v[idx])
                list_true_max_.append(counter[True]/tot_true_v)
                list_false_max_.append(counter[False]/tot_false_v)  

            axs.plot(array, list_true_max_, alpha=0.5)
            axs.plot(array, list_false_max_, label=f'{dim}', alpha=0.5)
            
            axs.legend()
            fig.suptitle(f'RF with {measure} n_clusters={n} layer={layer}\n', fontsize=12)
            # axs[j].set_title(f'weights={formatted_weight}')
            #axs[j,k].title(f'dim={dim} num_clusters={n}', fontsize=16)
            
            
fig.tight_layout()
fig.subplots_adjust(top=0.9)

In [ ]:
val, idx = torch.topk(cs, 5, axis=1)

In [ ]:
idx[:, 1]

In [ ]:
(v_labels['train']).to(int)

In [ ]:
torch.sum((v_labels['train']).to(int)==torch.argmax(cs, axis=1))

In [ ]:
torch.sum((v_labels['train']).to(int)==idx[:, 1])

## Testing with memmap


In [ ]:
import os
import json
import torch
from tensordict import TensorDict
from tensordict import MemoryMappedTensor as MMT
from datasets.cifar import Cifar
from models.model_wrap import ModelWrap 
from peepholes.peepholes import Peepholes
from clustering.clustering import Clustering
from clustering.clustering import prepare_data, compute_scores

In [ ]:
use_cuda = torch.cuda.is_available()
cuda_index = torch.cuda.device_count() - 2
device = torch.device(f"cuda:{cuda_index}" if use_cuda else "cpu")
print(f"Using {device} device")

#--------------------------------
# Parameters
#--------------------------------
dnn_model = 'vgg16'
abs_path = '/srv/newpenny/XAI/generated_data'
dataset = 'CIFAR100' 
seed = 29
bs = 64
ds = Cifar(dataset=dataset)

ds.load_data(batch_size=bs, data_kwargs={'num_workers': 4, 'pin_memory': True}, seed=seed) 
print('Loading the ex-peepholes')

phs_name = 'peepholes'
phs_dir = os.path.join(abs_path, 'peepholes')
peepholes = Peepholes(path=phs_dir, name=phs_name)
loaders = ds.get_dataset_loaders()

# Copy dataset to peepholes dataset
peepholes.get_peep_dataset(loaders=loaders, verbose=True) 
ph_dl = peepholes.get_dataloaders(batch_size=128, verbose=True)

available_layers = list(next(iter(ph_dl['train']))['peepholes'].keys())

In [ ]:
print('Computing confidence scores')

k_list = [20, 50, 70]
n_clusters_list = [50, 100, 150, 200]
layers_list = available_layers
algorithm = 'gmm'

# check for existing results or init results container
res_dir = 'clustering/confidence_scores'
res_path = os.path.join(abs_path, res_dir)
tensor_dict_path = os.path.join(res_path, f'algorithm={algorithm}_dataset={dataset}_dnn={dnn_model}.memmap')

if os.path.exists(tensor_dict_path):
    print('Results already present')
    all_scores = TensorDict.load_memmap(tensor_dict_path)
else:
    all_scores = TensorDict({}, batch_size=[])

In [ ]:
# converting my old data to memmap

# Ensure the results directory exists
res_dir = 'clustering/confidence_scores'
res_path = os.path.join('/srv/newpenny/XAI/generated_data', res_dir)

if not os.path.exists(res_path):
    os.makedirs(res_path)

# Loading existing scores from .pth file if needed
old_tensor_dict_path = os.path.join(res_path, 'algorithm=gmm_dataset=CIFAR100_dnn=vgg16.pth')

# Check if the file exists
if os.path.exists(old_tensor_dict_path):
    # Load existing all_scores
    all_scores = torch.load(old_tensor_dict_path)

    # Create a new MemoryMappedTensor with the same structure
    new_all_scores = TensorDict({}, batch_size=[])

    for k in all_scores.keys():
        new_all_scores.set(str(k), TensorDict({}, batch_size=[]))
        for n_clusters in all_scores[str(k)].keys():
            new_all_scores[str(k)].set(str(n_clusters), TensorDict({}, batch_size=[]))
            for split in all_scores[str(k)][str(n_clusters)].keys():
                new_all_scores[str(k)][str(n_clusters)].set(split, TensorDict({}, batch_size=[]))
                for layer in all_scores[str(k)][str(n_clusters)][split].keys():
                    # Create MemoryMappedTensor for max and entropy
                    _max = all_scores[str(k)][str(n_clusters)][split][layer]['max']
                    _entropy = all_scores[str(k)][str(n_clusters)][split][layer]['entropy']

                    new_all_scores[str(k)][str(n_clusters)][split][layer] = {
                        'max': MMT(_max.shape),
                        'entropy': MMT(_entropy.shape)
                    }
                    # Populate the MMT with existing data
                    new_all_scores[str(k)][str(n_clusters)][split][layer]['max'].copy_(_max)
                    new_all_scores[str(k)][str(n_clusters)][split][layer]['entropy'].copy_(_entropy)

    # Save the new MemoryMappedTensor
    new_tensor_dict_path = os.path.join(res_path, 'algorithm=gmm_dataset=CIFAR100_dnn=vgg16.memmap')
    new_all_scores.memmap(new_tensor_dict_path, num_threads=4)

    print(f"Converted and saved new all_scores with memory mapping at {new_tensor_dict_path}.")
else:
    print(f"No existing scores found at {old_tensor_dict_path}. Please check the path.")


In [ ]:
# init the peephole container if not existing
new_peep_dir = 'clustering/peepholes' 
new_peep_path = os.path.join(abs_path, new_peep_dir) 
new_peep_tensor_dict_path = os.path.join(new_peep_path, f'algorithm={algorithm}_dataset={dataset}_dnn={dnn_model}.memmap')

if os.path.exists(new_peep_tensor_dict_path):
    print('New peepholes results already present')
    peephole_scores = TensorDict.load_memmap(new_peep_tensor_dict_path)
else:
    print('Initializing peephole container')
    peephole_scores = TensorDict({}, batch_size=[])

In [ ]:
peephole_scores

In [ ]:
# Loop over k and n_clusters
for k in k_list:
    str_k = str(k)
    if str_k not in all_scores.keys():
        all_scores.set(str_k, TensorDict({}, batch_size=[]))
    
    for n_clusters in n_clusters_list:
        str_n_clusters = str(n_clusters)

        # Check if the combination of k and n_clusters already exists in the scores
        if str_n_clusters in all_scores[str_k].keys():
            existing_layers = all_scores[str_k][str_n_clusters]['train'].keys()
            
            # Compute only the missing layers
            for layer in layers_list:
                if layer not in existing_layers:
                    data = prepare_data(ph_dl, [layer], k)
                    print(f'Clustering for layer={layer}')
                    compute_scores(k, n_clusters, algorithm, [layer], data, all_scores, metadata)
        else:
            print(f'Clustering with algorithm={algorithm}, k={k}, n_clusters={n_clusters}')
            # If not existing, create the subdict for n_clusters
            all_scores[str_k].set(str_n_clusters, TensorDict({
                'train': TensorDict({}, batch_size=[]),
                'val': TensorDict({}, batch_size=[])
            }, batch_size=[]))
            
            # Prepare data for all splits
            data = prepare_data(ph_dl, layers_list, k)
            compute_scores(k, n_clusters, algorithm, layers_list, data, all_scores, metadata)

# Pre-allocate memory-mapped tensors for scores
for layer in layers_list:
    n_samples = len(loaders['train'].dataset)  # Get the number of samples from the training loader
    all_scores[str_k][str_n_clusters]['train'][layer] = MMT.empty(shape=torch.Size((n_samples,)))  # Initialize with MMT

# Save all scores and metadata after processing
all_scores.memmap(tensor_dict_path, num_threads=4)  # Specify the number of threads for saving
with open(meta_path, 'w') as json_file:
    json.dump(metadata, json_file, indent=4)

print('Results and metadata saved.')

In [ ]:
peephole_scores[str(k)][str(n_clusters)][split]['features.24']

In [ ]:
torch.max(peephole_scores[str(k)][str(n_clusters)][split]['classifier.3'], axis=1)

In [ ]:
data['true_labels']['val']

In [ ]:
c = data['true_labels']['val']
_, g = torch.max(peephole_scores[str(50)][str(150)][split]['classifier.0'], axis=1)

In [ ]:
torch.sum(c == g)

In [ ]:
for k in k_list:  # loop over core-vector dimension
    str_k = str(k)

    # initialize peephole_scores for k
    if str_k not in peephole_scores.keys():
        peephole_scores.set(str_k, TensorDict({}, batch_size=[]))

    for n_clusters in n_clusters_list:  # loop over n_clusters
        str_n_clusters = str(n_clusters)

        # initialize peephole_scores for n_clusters
        if str_n_clusters not in peephole_scores[str_k].keys():
            peephole_scores[str_k].set(str_n_clusters, TensorDict({
                'train': TensorDict({}, batch_size=[]),
                'val': TensorDict({}, batch_size=[]),
            }, batch_size=[]))

        # for both train and val splits, ensure layers exist
        for split in ['train', 'val']:
            if split not in peephole_scores[str_k][str_n_clusters].keys():
                peephole_scores[str_k][str_n_clusters].set(split, TensorDict({}, batch_size=[]))

            existing_layers = peephole_scores[str_k][str_n_clusters][split].keys()
            
            # check if all layers are present in the current split
            if existing_layers and any(layer not in existing_layers for layer in layers_list):
                for layer in layers_list:
                    if layer not in existing_layers:
                        data = prepare_data(ph_dl, [layer], k)
                        print(f'Clustering for layer={layer} with n_clusters={n_clusters}')
                        compute_scores(k, 
                                       n_clusters, 
                                       algorithm, 
                                       [layer], 
                                       data, 
                                       peephole_scores, 
                                       all_scores, 
                                       compute_scores=True, 
                                       seed=42)

            else:
                print(f'Skipping {algorithm} k={k}, n_clusters={n_clusters} for {split} layers')
                continue  # skip to the next n_clusters if all layers have data

        # if not all layers have data, we still need to compute scores
        if str_n_clusters not in peephole_scores[str_k].keys() or not existing_layers:
            print('Clustering')
            print(f'algorithm={algorithm}, k={k}, n_clusters={n_clusters}')

            data = prepare_data(ph_dl, layers_list, k)

            n_samples_train = len(data['core_vectors']['train'][layers_list[0]])
            n_samples_val = len(data['core_vectors']['val'][layers_list[0]])

            # initialize all_scores for n_clusters
            all_scores[str_k].set(str_n_clusters, TensorDict({
                'train': TensorDict({
                    layer: MMT.empty(shape=(n_samples_train,)) for layer in layers_list}, batch_size=[]),
                'val': TensorDict({
                    layer: MMT.empty(shape=(n_samples_val,)) for layer in layers_list}, batch_size=[])
            }, batch_size=[]))

            compute_scores(k, 
                           n_clusters, 
                           algorithm, 
                           layers_list, 
                           data, 
                           peephole_scores, 
                           all_scores, 
                           compute_scores=True, 
                           seed=42)

# save results
peephole_scores.memmap(new_peep_tensor_dict_path, num_threads=4)
all_scores.memmap(tensor_dict_path, num_threads=4)
print('Results saved to memory-mapped tensor.')


In [ ]:
layer_dict = TensorDict({layer: MMT.empty(shape=(n_samples, n_classes)) for layer in layers_list}, batch_size=[])

peephole_scores[str(k)].set(str(n_clusters), TensorDict({
    split: layer_dict  # Pre-allocate memory-mapped tensor for each layer
}, batch_size=[]))

## stuff with coreVectors

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "6, 7"
os.environ['SCIPY_USE_PROPACK'] = "True"
 
threads = "64"
os.environ["OMP_NUM_THREADS"] = threads
os.environ["OPENBLAS_NUM_THREADS"] = threads
os.environ["MKL_NUM_THREADS"] = threads
os.environ["VECLIB_MAXIMUM_THREADS"] = threads
os.environ["NUMEXPR_NUM_THREADS"] = threads

In [ ]:
# python stuff
from pathlib import Path as Path
from numpy.random import randint

# Our stuff
from datasets.cifar import Cifar
from models.model_wrap import ModelWrap 
from coreVectors.coreVectors import CoreVectors 
from coreVectors.svd_coreVectors import reduct_matrices_from_svds as parser_fn

In [ ]:
# torch stuff
import torch
from torchvision.models import vgg16, VGG16_Weights

In [ ]:
from classifier.classifier_base import trim_corevectors
from classifier.kmeans import KMeans 
from classifier.gmm import GMM 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from peepholes.peepholes import Peepholes

In [ ]:
import pandas as pd
import numpy as np

def evaluate_kmeans(layer_list, n_clusters_list, peep_size_list, n_classes, ph_dl, true_labels, return_preds=False):
    results = [] 
    predictions = []
    
    for peep_size in peep_size_list:
        for n_clusters in n_clusters_list:
            print(f'KMeans')
        
            for layer in layer_list:
                random_seed = np.random.randint(0, 2**32-1)
                print(f'Layer: {layer}, Peep size: {peep_size}, seed={random_seed}')
                
                parser_kwargs = {'layer': layer, 'peep_size': peep_size}
                cls_kwargs = {'random_state': random_seed, 'n_init': n_classes, 'max_iter': 500}
                
           
                cls = KMeans(
                    nl_classifier=n_clusters,
                    nl_model=n_classes,
                    parser=parser_cv,
                    parser_kwargs=parser_kwargs,
                    cls_kwargs=cls_kwargs
                )
                
                cls.fit(dataloader=ph_dl['train'], verbose=False)
                cls.compute_empirical_posteriors(verbose=False)
                
                #plt.imshow(cls._empp)
                
                peepholes = Peepholes(classifier=cls)
                peepholes.get_peepholes(dataloader=ph_dl['val'], verbose=False)
                preds = peepholes._phs
                
                acc = torch.sum(torch.argmax(preds, axis=1) == true_labels['val']).item() / len(true_labels['val'])
                print(f'Layer: {layer}, Peep size: {peep_size}, Clusters: {n_clusters} Accuracy: {acc}')
                
                results.append({'layer': layer, 'n_clusters': n_clusters, 'peep_size': peep_size, 'accuracy': acc})
                predictions.append({'layer': layer, 'n_clusters': n_clusters, 'peep_size': peep_size, 
                                    'labels_pred': torch.argmax(preds, axis=1)})
    
    df = pd.DataFrame(results)
    df.set_index(['layer', 'n_clusters', 'peep_size'], inplace=True)

    df_pred = pd.DataFrame(predictions)
    df_pred.set_index(['layer', 'n_clusters', 'peep_size'], inplace=True)

    if return_preds:
        return df, df_pred
    else:
        return df

In [ ]:
def evaluate_gmm(layer_list, n_clusters_list, peep_size_list, n_classes, ph_dl, true_labels, return_preds=False):
    results = [] 
    predictions = []
    
    for peep_size in peep_size_list:
        for n_clusters in n_clusters_list:
            print(f'GMM')
        
            for layer in layer_list:
                random_seed = np.random.randint(0, 2**32-1)
                print(f'Layer: {layer}, Peep size: {peep_size}, seed={random_seed}')
                
                parser_kwargs = {'layer': layer, 'peep_size': peep_size}
                cls_kwargs = {'random_state': random_seed, 'n_init': n_classes, 'max_iter': 500}
               
           
                cls = GMM(
                    nl_classifier=n_clusters,
                    nl_model=n_classes,
                    parser=parser_cv,
                    parser_kwargs=parser_kwargs,
                    cls_kwargs=cls_kwargs
                )
                
                cls.fit(dataloader=ph_dl['train'], verbose=False)
                cls.compute_empirical_posteriors(verbose=False)
                
                #plt.imshow(cls._empp)
                
                peepholes = Peepholes(classifier=cls)
                peepholes.get_peepholes(dataloader=ph_dl['val'], verbose=False)
                preds = peepholes._phs
                
                acc = torch.sum(torch.argmax(preds, axis=1) == true_labels['val']).item() / len(true_labels['val'])
                print(f'Layer: {layer}, Peep size: {peep_size}, Clusters: {n_clusters} Accuracy: {acc}')
                
                results.append({'layer': layer, 'n_clusters': n_clusters, 'peep_size': peep_size, 'accuracy': acc})
                predictions.append({'layer': layer, 'n_clusters': n_clusters, 'peep_size': peep_size, 
                                    'labels_pred': torch.argmax(preds, axis=1)})
    
    df = pd.DataFrame(results)
    df.set_index(['layer', 'n_clusters', 'peep_size'], inplace=True)

    df_pred = pd.DataFrame(predictions)
    df_pred.set_index(['layer', 'n_clusters', 'peep_size'], inplace=True)

    if return_preds:
        return df, df_pred
    else:
        return df

In [ ]:
use_cuda = torch.cuda.is_available()
cuda_index = torch.cuda.device_count() - 2
device = torch.device(f"cuda:{cuda_index}" if use_cuda else "cpu")
print(f"Using {device} device")

In [ ]:
#--------------------------------
# Dataset 
#--------------------------------
# model parameters
dataset = 'CIFAR100' 
seed = 29
bs = 64

ds_path = f'/srv/newpenny/dataset/{dataset}'
ds = Cifar(data_path=ds_path,
           dataset=dataset)
ds.load_data(
        batch_size = bs,
        data_kwargs = {'num_workers': 4, 'pin_memory': True},
        seed = seed,
        )

#--------------------------------
# Model 
#--------------------------------
pretrained = True
model_dir = '/srv/newpenny/XAI/models'
if dataset=='CIFAR100':
    # CIFAR100
    model_name = f'vgg16_pretrained={pretrained}_dataset={dataset}-'\
                 f'augmented_policy=CIFAR10_bs={bs}_seed={seed}.pth'
elif dataset=='CIFAR10':
    # CIFAR10
    model_name = f'_vgg16_pretrained={pretrained}_dataset={dataset}-'\
                 f'augmented_policy=CIFAR10_seed={seed}.pth'

nn = vgg16(weights=VGG16_Weights.IMAGENET1K_V1)
in_features = 4096
num_classes = len(ds.get_classes())
nn.classifier[-1] = torch.nn.Linear(in_features, num_classes)
model = ModelWrap(device=device)
model.set_model(model=nn, path=model_dir, name=model_name, verbose=True)

# layers_dict = {'classifier': [0, 3],
#                'features': [24, 28]}
layers_dict = {'classifier': [0, 3],
               'features': [14, 24, 28]}

model.set_target_layers(target_layers=layers_dict, verbose=True)
print('target layers: ', model.get_target_layers()) 


#--------------------------------
# CoreVectors 
#--------------------------------
phs_name = 'corevectors'
phs_dir = f'/srv/newpenny/XAI/generated_data/corevectors/{dataset}'
# phs_dir = Path.cwd()/'../data/corevectors'
cv = CoreVectors(
        path = phs_dir,
        name = phs_name,
        )

loaders = ds.get_dataset_loaders()

In [ ]:
# copy dataset to coreVect dataset
cv.get_coreVec_dataset(
        loaders = loaders,
        verbose = True
        ) 

# cv.get_activations(
#         model=model,
#         loaders=loaders,
#         verbose=True
#         )

# cv.get_coreVectors(
#         model = model,
#         reduct_matrices = model._svds,
#         parser = parser_fn,
#         verbose = True
#         )

ph_dl = cv.get_dataloaders(batch_size=3, verbose=True)

## Classifier

### KMeans

In [ ]:
n_classes = 10 if dataset=='CIFAR10' else 100
n_classes

In [ ]:
layer = 'classifier.0'
n_clusters = 150
peep_size = 10

In [ ]:
parser_cv = trim_corevectors
parser_kwargs = {'layer': layer, 'peep_size':peep_size}
cls_kwargs = {'random_state': 42, 'n_init':n_classes, 'max_iter':500} 

cls = KMeans(
        nl_classifier = n_clusters,
        nl_model = n_classes,
        parser = parser_cv,
        parser_kwargs = parser_kwargs,
        cls_kwargs = cls_kwargs
        )

In [ ]:
cls.fit(dataloader = ph_dl['val'], verbose=True)
cls.compute_empirical_posteriors(verbose=True)

In [ ]:
pl = cls.classifier_probabilities(dataloader=ph_dl['val'])

In [ ]:
cls._classifier_probs[0]

In [ ]:
_dl = ph_dl['val']
n_samples = len(_dl.dataset)
bs = _dl.batch_size

# Allocate predict probs
_classifier_probs = torch.zeros(n_samples, 100)

for bn, batch in enumerate(tqdm(_dl)):
    data = cls.parser(data = batch, **cls.parser_kwargs)
    n_in = len(data)
    
    distances = torch.tensor(cls._classifier.transform(data))

    # convert distances to probabilities (soft assignment) Gaussian-like softmax
    # TODO: Check the var in the exponent. Should it be over the training set? Should it be there?
    #probs = torch.exp(-distances ** 2 / (2 * (distances.std() ** 2)))
    probs = torch.exp(-distances ** 2 / 2 )

    # normalize to probabilities
    probs /= probs.sum(dim=1, keepdim=True)
    
    _classifier_probs[bn*bs:bn*bs+n_in] = probs

In [ ]:
distances

In [ ]:
_classifier_probs[0]

In [ ]:
coeff = 1 / ((2 * np.pi) ** (20 / 2))
coeff * torch.exp(-0.5 * distances[0]**2)

In [ ]:
cls._empp.double() @ torch.exp(-0.5 * distances[0]**2)

In [ ]:
torch.sort(torch.argmax(cls._empp, axis=1))

In [ ]:
cls._classifier.predict(data)

In [ ]:
plt.imshow(cls._empp)

In [ ]:
peepholes = Peepholes(classifier=cls)

In [ ]:
peepholes.get_peepholes(dataloader=ph_dl['val'], verbose=True)

In [ ]:
preds = peepholes._phs

In [ ]:
torch.argmax(preds, axis=1)

In [ ]:
torch.sum(torch.argmax(preds, axis=1)==true_labels['val'])

In [ ]:
k_list = [5, ]
n_list = [10, 20, 30, 50]

df = evaluate_gmm(layer_list, n_list, k_list, n_classes, ph_dl, true_labels)

In [ ]:
layer_list = ['features.14', 'features.24', 'features.28', 'classifier.0', 'classifier.3']

In [ ]:
k_list = [5, 9, 50]
n_list = [50, 100, 150] #[10, 20, 30, 50, 70]

df_k, preds_k = evaluate_kmeans(layer_list, n_list, k_list, n_classes, ph_dl, true_labels, return_preds=True)

### GMM

In [ ]:
n_classes = 10 if dataset=='CIFAR10' else 100
n_classes

In [ ]:
layer = 'classifier.0'
n_clusters = 100
peep_size = 10

In [ ]:
parser_cv = trim_corevectors
parser_kwargs = {'layer': layer, 'peep_size':peep_size}
cls_kwargs = {'random_state': 42, 'n_init':n_classes, 'max_iter':500} 

cls = GMM(
        nl_classifier = n_clusters,
        nl_model = n_classes,
        parser = parser_cv,
        parser_kwargs = parser_kwargs,
        cls_kwargs = cls_kwargs
        )

In [ ]:
cls.fit(dataloader = ph_dl['train'], verbose=True)
cls.compute_empirical_posteriors(verbose=True)

In [ ]:
cls.classifier_probabilities(dataloader=ph_dl['val'])

In [ ]:
cls._classifier_probs[0]

In [ ]:
_dl = ph_dl['val']
n_samples = len(_dl.dataset)
bs = _dl.batch_size

# Allocate predict probs
_classifier_probs = torch.zeros(n_samples, 100)

for bn, batch in enumerate(tqdm(_dl)):
    data = cls.parser(data = batch, **cls.parser_kwargs)
    n_in = len(data)

    probs = torch.tensor(cls._classifier.predict_proba(data))
    _classifier_probs[bn*bs:bn*bs+n_in] = probs

In [ ]:
_classifier_probs[0]

In [ ]:
coeff = 1 / ((2 * np.pi) ** (20 / 2))
coeff * torch.exp(-0.5 * distances[0]**2)

In [ ]:
cls._empp.double() @ torch.exp(-0.5 * distances[0]**2)

In [ ]:
torch.sort(torch.argmax(cls._empp, axis=1))

In [ ]:
cls._classifier.predict(data)

In [ ]:
plt.imshow(cls._empp)

In [ ]:
peepholes = Peepholes(classifier=cls)

In [ ]:
peepholes.get_peepholes(dataloader=ph_dl['val'], verbose=True)

In [ ]:
preds = peepholes._phs

In [ ]:
torch.argmax(preds, axis=1)

In [ ]:
torch.sum(torch.argmax(preds, axis=1)==true_labels['val'])

In [ ]:
k_list = [5, ]
n_list = [10, 20, 30, 50]

df = evaluate_gmm(layer_list, n_list, k_list, n_classes, ph_dl, true_labels)

In [ ]:
layer_list = ['features.14', 'features.24', 'features.28', 'classifier.0', 'classifier.3'] # CIFAR100

In [ ]:
k_list = [5, 9, 50, 100]
n_list = [100, 150, 200] #[10, 20, 30, 50, 70]

df_g, preds_g = evaluate_gmm(layer_list, n_list, k_list, n_classes, ph_dl, true_labels, return_preds=True)

### results with CIFAR100

In [ ]:
df_k.unstack(level='layer').unstack(level='peep_size')

In [ ]:
preds_k.info()

In [ ]:
preds_k.xs(('classifier.3', 150, 50)).values

In [ ]:
from collections import Counter

In [ ]:
np.sum(x==tl)/len(tl)

In [ ]:
tl = np.array(true_labels['val'])[decisions['val']]
true_count = Counter(tl)
true_unique_labels = list(true_count.keys())
true_frequencies = list(true_count.values())

In [ ]:
len(x)

In [ ]:
x = np.array(preds_k.xs(('features.14', 100, 5)).values[0])[decisions['val']]

label_counts = Counter(x)

unique_labels = list(label_counts.keys())
frequencies = list(label_counts.values())

In [ ]:
n = 150 # fix n_clusters
algo = 'KMeans'

fig, axs = plt.subplots(len(layer_list), len(k_list), sharex=True, figsize=(12, 12))

title_ = f'Pred vs True labels - clustering={algo}, n_clusters={n}, dataset={dataset}'
fig.suptitle(title_)

for i, layer in enumerate(layer_list):
    axs[i, 0].set_ylabel(layer)
    for j, k in enumerate(k_list):

        axs[0, j].set_title(f'corevector_size={k}')

        x = np.array(preds_k.xs((layer, n, k)).values[0])[decisions['val']]

        label_counts = Counter(x)
        
        unique_labels = list(label_counts.keys())
        frequencies = list(label_counts.values())

        acc_correct = np.sum(x==tl)/len(tl)

        axs[i, j].bar(unique_labels, frequencies, alpha=0.5, label=f'acc={acc_correct:.4f}')
        axs[i, j].bar(true_unique_labels, true_frequencies, alpha=0.5)

        axs[i, j].legend()

plt.tight_layout()
plt.savefig(title_+'.png')

### results with CIFAR10

In [ ]:
df.unstack(level='layer')

In [ ]:
print('seed=42')
df_k.unstack(level='layer').unstack(level='peep_size')

In [ ]:
print('Random seeds')
df_k2.unstack(level='layer').unstack(level='peep_size')

## Caccia le immaginine

In [ ]:
from clustering.utils import get_unique_values

In [ ]:
abs_path = '/srv/newpenny/XAI/generated_data'

In [ ]:
algorithm = 'gmm'
dataset = 'CIFAR100'
dnn_model = 'vgg16'

### Load peephole scores

In [ ]:
algorithm = 'gmm'
# init the peephole container if not existing
new_peep_dir = 'clustering/peepholes' 
new_peep_path = os.path.join(abs_path, new_peep_dir) 
new_peep_tensor_dict_path = os.path.join(new_peep_path, f'algorithm={algorithm}_dataset={dataset}_dnn={dnn_model}.memmap')

if os.path.exists(new_peep_tensor_dict_path):
    print('New peepholes results already present')
    peephole_scores = TensorDict.load_memmap(new_peep_tensor_dict_path)
else:
    print('Initializing peephole container')
    peephole_scores = TensorDict({}, batch_size=[])

ps_gmm = peephole_scores

In [ ]:
algorithm = 'kmeans'
# init the peephole container if not existing
new_peep_dir = 'clustering/peepholes' 
new_peep_path = os.path.join(abs_path, new_peep_dir) 
new_peep_tensor_dict_path = os.path.join(new_peep_path, f'algorithm={algorithm}_dataset={dataset}_dnn={dnn_model}.memmap')

if os.path.exists(new_peep_tensor_dict_path):
    print('New peepholes results already present')
    peephole_scores = TensorDict.load_memmap(new_peep_tensor_dict_path)
else:
    print('Initializing peephole container')
    peephole_scores = TensorDict({}, batch_size=[])

ps_kmeans = peephole_scores

In [ ]:
scores = {'results_tensordict' : peephole_scores}
hps = get_unique_values(**scores)

In [ ]:
k_list, n_list, splits, layers = hps

### Get true labels

In [ ]:
splits = ['val']

In [ ]:
from tqdm import tqdm

In [ ]:
true_labels = {}
decisions = {}

for split in splits:
    true_labels[split] = []
    decisions[split] = []

    for batch in tqdm(ph_dl[split]):
        peepholes = batch['coreVectors']
        labels = batch['label']
        decision_results = batch['result']

        true_labels[split].append(labels)
        decisions[split].append(decision_results.bool())

    true_labels[split] = torch.cat(true_labels[split], dim=0)
    decisions[split] = torch.cat(decisions[split], dim=0)

### Check accuracy

In [ ]:
import pandas as pd

In [ ]:
tl==lp

In [ ]:
acc = {}
idxs = {}

for k in tqdm(k_list):
    for n in n_list:
        for split in splits:
            tl = true_labels[split]
            
            for l in layers:
                lp = torch.argmax(peephole_scores[k][n][split][l], axis=1)
                res = np.float64(torch.sum(tl==lp) / len(tl))

                acc[(k, n, split, l)] = res

                eq_idx = torch.nonzero(tl==lp, as_tuple=False).squeeze()
                idxs[(k, n, split, l)] = eq_idx

In [ ]:
# solo val 
split = 'val'
tl = true_labels[split]
peephole_scores = ps_gmm
acc_gmm = {}
top_acc_gmm = {}

idxs_gmm = {}
top_idxs_gmm = {}

for k in tqdm(k_list):
    for n in n_list:            
            for l in layers:
                scores = ps_gmm[k][n][split][l]
                
                lp = torch.argmax(scores, axis=1)
                # std acc
                res = np.float64(torch.sum(tl==lp) / len(tl))
                acc_gmm[(k, n, l)] = res
                eq_idx = torch.nonzero(tl==lp, as_tuple=False).squeeze()
                idxs_gmm[(k, n, l)] = eq_idx
                # top acc
                _, top_idxs = torch.topk(scores, 10, dim=1)
                top_correct = (tl.unsqueeze(1)==top_idxs).any(dim=1)
                top_acc = top_correct.float().mean().item()
                top_idxs_gmm[(k, n, l)] = top_acc

                

In [ ]:
top_idxs_gmm

In [ ]:
# solo val 
split = 'val'
tl = true_labels[split]
peephole_scores = ps_kmeans
acc_kmeans = {}
idxs_kmeans = {}

for k in tqdm(k_list):
    for n in n_list:            
            for l in layers:
                scores = ps_kmeans[k][n][split][l]
                lp = torch.argmax(scores, axis=1)
                res = np.float64(torch.sum(tl==lp) / len(tl))

                acc_kmeans[(k, n, l)] = res

                eq_idx = torch.nonzero(tl==lp, as_tuple=False).squeeze()
                idxs_kmeans[(k, n, l)] = eq_idx

In [ ]:
true_labels['val']

In [ ]:
n_list

In [ ]:
layers = [
    'features.7',
    'features.14',
     'features.24',
     'features.26',
     'features.28',
     'classifier.0',
     'classifier.3', ]

In [ ]:
# idx_prova = 9478
idx_prova = 151
k_ = k_list[-1]
n_ = n_list[2]
split = 'val'

t1 = []
t2 = []
t3 = []

for l in layers:
    # tensor = ps_kmeans[k_][n_][split][l][idx_prova]
    # if idx_prova in idxs_kmeans[(k_, n_, l)]:
    tensor = ps_gmm[k_][n_][split][l][idx_prova]
    if idx_prova in idxs_gmm[(k_, n_, l)]:
        print(f'{idx_prova} present for {l}')
    t1.append(tensor)
    t2.append(tensor**0.5)
    t3.append(tensor**2)

pc1 = torch.stack(t1)
pc2 = torch.stack(t2)
pc3 = torch.stack(t3)

# pc.shape

In [ ]:
fig, axs = plt.subplots(1, 6, sharey=True, figsize=(8, 8))

fig.suptitle(f"Combined peepholes; instance_idx={idx_prova}, true_label={true_labels['val'][idx_prova]}")

axs[0].imshow(pc1.T)
axs[0].set_title('exp=1')
axs[1].imshow(torch.sum(pc1, axis=0).reshape(-1, 1))

axs[2].imshow(pc2.T)
axs[2].set_title('exp=0.5')
axs[3].imshow(torch.sum(pc2, axis=0).reshape(-1, 1))

axs[4].imshow(pc3.T)
axs[4].set_title('exp=2')
axs[5].imshow(torch.sum(pc3, axis=0).reshape(-1, 1))

plt.show()

In [ ]:
true_labels['val'][idx_prova]

In [ ]:
idxs_kmeans[(k, n, layers[2])]

In [ ]:
k, n, layers

In [ ]:
from collections import Counter

In [ ]:
k_ = k_list[0]
n_ = n_list[0]
l_ = 'classifier.0'

idx_counter = Counter()

for key, indices in idxs_kmeans.items():
    k, n, l = key
    # if k==k_ and n==n_:
    if l==l_:
        idx_counter.update(list(indices))

num_configs = sum(1 for key in idxs_kmeans.keys() if key[2]==l_)#if key[0]==k_ and key[1]==n_)

common_idx = [index for index, count in idx_counter.items() if count==num_configs]

In [ ]:
len(idxs_kmeans[(k_, n_, l_)])

In [ ]:
len(idxs_gmm[(k_, n_, l_)])

In [ ]:
a = idxs_kmeans[(k_, n_, layers[0])]
b = idxs_kmeans[(k_, n_, layers[1])]

cmn = []

for lb in a:
    if lb in b:
        cmn.append(lb)

In [ ]:
len(cmn)

In [ ]:
a = idxs_kmeans[(k_, n_, layers[2])]
b = idxs_kmeans[(k_, n_, layers[3])]

cmn = []

for lb in a:
    if lb in b:
        cmn.append(lb)

In [ ]:
cmn

In [ ]:
a = idxs_kmeans[(k_, n_, layers[3])]
b = idxs_kmeans[(k_, n_, layers[4])]

cmn = []

for lb in a:
    if lb in b:
        cmn.append(lb)

In [ ]:
cmn

In [ ]:
a = idxs_kmeans[(k_, n_, layers[4])]
b = idxs_kmeans[(k_, n_, layers[5])]

cmn = []

for lb in a:
    if lb in b:
        cmn.append(lb)

In [ ]:
cmn

In [ ]:
a = idxs_kmeans[(k_, n_, layers[4])]
b = idxs_kmeans[(k_, n_, layers[6])]

cmn = []

for lb in a:
    if lb in b:
        cmn.append(lb)

In [ ]:
a

In [ ]:
acc_series = pd.Series(acc_kmeans)
acc_df_kmeans = acc_series.to_frame(name='accuracy')
# acc_df.index.names = ['k', 'n', 'split', 'l']
acc_df_kmeans.index.names = ['k', 'n', 'l']
acc_df_kmeans = acc_df_kmeans.unstack(level='l')

In [ ]:
idxmax_kmeans = pd.Series(acc_df_kmeans.idxmax())
idxmax_kmeans['accuracy', 'classifier.0']

In [ ]:
acc_df_kmeans.style.background_gradient(cmap='YlOrRd', axis=1)

In [ ]:
acc_series = pd.Series(acc_gmm)
acc_df_gmm = acc_series.to_frame(name='accuracy')
# acc_df.index.names = ['k', 'n', 'split', 'l']
acc_df_gmm.index.names = ['k', 'n', 'l']
acc_df_gmm = acc_df_gmm.unstack(level='l')

In [ ]:
idxmax_gmm = pd.Series(acc_df_gmm.idxmax())
idxmax_gmm['accuracy', 'classifier.0']

In [ ]:
acc_df_gmm.style.background_gradient(cmap='YlOrRd', axis=1)

In [ ]:
acc_df_unstacked.loc[('100', '100')]

In [ ]:
peephole_scores[k][n]['val'][l]